In [2]:
import pandas as pd
import pprint
import numpy as np
import collections
import sklearn.preprocessing as preprocessing
from IPython.core.debugger import Tracer
import json

In [217]:
dados_json = pd.read_json("dados.json")

# columns = reversed(dados_json.columns)
dados_json['user_id'] = range(0,len(dados_json))
dados_json = dados_json.reindex_axis(['user_id','tipo_user','habilidades','segmento','horas_disponiveis'], axis=1) #reordenamos somente pra melhorar a visualização e ficar igual csv


In [218]:
classes = set([x.strip() for y in dados_json['habilidades'] for x in y])
classes

{'angular',
 'big-data',
 'css',
 'html',
 'illustrator',
 'ionic',
 'js',
 'laravel',
 'machine-learning',
 'photoshop',
 'php',
 'python',
 'scikit-learn',
 'scrum',
 'scss'}

In [40]:
dados_json = pd.read_json("dados.json")

# columns = reversed(dados_json.columns)
dados_json['user_id'] = range(0,len(dados_json))
dados_json = dados_json.reindex_axis(['user_id','tipo_user','habilidades','segmento','horas_disponiveis'], axis=1) #reordenamos somente pra melhorar a visualização e ficar igual csv

classes = set([x.strip() for y in dados_json['habilidades'] for x in y])
#aqui ainda nao existe uma coluna pra cada classe
#criando as colunas com o nome das classes
for classe in classes: dados_json[classe] = 0
    
# # Modifying columns to '1' when habilidade is found
   
for i in range(len(dados_json['habilidades'])): # para cada linha, ou seja, cada usuario
    classes_line = dados_json.loc[i,'habilidades'] #pegamos suas habilidades
#     print(classes_line);
#     print("-------------")
    for j in range(len(classes_line)): #para cada habilidade daquele usuario
        classe = classes_line[j] #pegamos a atual habilidade do usuario
        if classe in classes: #verificamos se a habilidade atual existe no nosso vetor de classes global
            dados_json.loc[:, (classe)][i] = 1
              
del dados_json['habilidades']
dados_json.head()
# print(classes)
# dados_json

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,user_id,tipo_user,segmento,horas_disponiveis,big-data,js,html,ionic,scss,laravel,scikit-learn,python,scrum,css,angular,illustrator,machine-learning,php,photoshop
0,0,startup,fintech,6,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0
1,1,user,edtech,8,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
2,2,startup,biotech,4,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0
3,3,startup,adtech,6,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
4,4,startup,adtech,8,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0


In [36]:
# dados_json.loc[:, ('habilidades')][3]

['scrum', 'laravel', 'angular', 'html']

In [38]:
#Conferindo formato csv
dados_csv = pd.read_csv("dados.csv",sep=";") 
dados_csv.head()

,tipo_user,habilidades,segmento,horas_disponiveis
0,startup,"python,php,js,scrum",fintech,6
1,user,"python,js,illustrator",edtech,8
2,startup,"laravel, php, js, html",biotech,4
3,startup,"scrum,laravel,angular,html",adtech,6
4,startup,"js,html,css",adtech,8


In [39]:
dados_json.head()

,user_id,tipo_user,habilidades,segmento,horas_disponiveis,big-data,js,html,ionic,scss,laravel,scikit-learn,python,scrum,css,angular,illustrator,machine-learning,php,photoshop
0,0,startup,"[python, php, js, scrum]",fintech,6,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0
1,1,user,"[python, js, illustrator]",edtech,8,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
2,2,startup,"[laravel, php, js, html]",biotech,4,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0
3,3,startup,"[scrum, laravel, angular, html]",adtech,6,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0
4,4,startup,"[js, html, css]",adtech,8,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0


In [212]:
class Ranking (object):

    def __init__(self,df,preprocessed=True):
        self.df = df
        
    def getDf (self):
        return self.df
    
    def preprocessDataFrame (self):# testar se é preciso receber df como parametro
        categorical_columns = self.df.select_dtypes(['object']).columns
        label_encoder = preprocessing.LabelEncoder()
        for column in categorical_columns:
            label_encoder.fit(self.df[column])
            label_encoder.classes_
            self.df[column] = label_encoder.transform(self.df[column])
    
    def train_test_split (self,test_size=0.6):
        limit = int((len(self.df)*test_size))
        X_treino = self.df[:limit]
        X_teste = self.df[limit:].reset_index(drop=True)
        return X_treino, X_teste
    
    
    '''
        Função que gera um dicionario de rankings dado um conjunto de teste(à ser recomendado) 
        e um conjunto de treinamento para realizar a recomendação
    '''
    # Do jeito que rank_the_data está hoje, estamos sujeitos à ter que fazer manutenção constante no código
    # Somente na parte dos pesos, talvez tornamos ele um dicionario cuja chave seja o nome da coluna com seu valor
    # o peso respectivo => TODO <=
    def rank_the_data(self,X_treino,X_teste, name_treino='Ranking_Treino', name_teste='Ranking_Teste'):

        temp_matches = {}
        matches_dict = {}
        
        for row in X_teste.itertuples(name=name_teste): # definindo a row atual do x_teste
            match = False
            valores_teste  = []
            valores_treino = []
            rating = 0
            pesos  = (len(row) - 3) * [0.0] #array com os pesos

            pesos[0] = 0.8 # aumentando peso do segmento para quando houver match

            for i in range(2,len(pesos)):
                pesos[i] = 1.0 # aumentando o peso das habilidades
            # para a disponibilidade de horas nós temos que fazer um calculo mais detalhado
            # diponibilidade_startup > disponibilidade_user; disponibilidade_user/disponibilidade_startup => peso
            # diponibilidade_startup < disponibilidade_user; disponibilidade_startup/disponibilidade_user => peso

            for i in range(3,len(row)):  
            # aqui é o loop do item da linha (colunas) começamos do index 2 por causa da estrutura da tupla
                valores_teste.append(row[i])

            for row_treino in X_treino.itertuples(name=name_treino):# loop da row atual do x_treino
                for i_treino in range(3, len(row_treino)): # loop dos itens da linha (colunas)
                    valores_treino.append(row_treino[i_treino]) # valor da coluna

                if not(row.tipo_user == row_treino.tipo_user): # nao podemos computar valores para usuarios de mesmo tipo
                    # para adicionar funcionalidade de peso teremos que modificar a partir da current_sum
                    # pensamento: e se já de inicio nós tivermos um array com todos os pesos settados com 1 e fizermos
                    # a multiplicação

                    #peso disp_horas
                    if row.horas_disponiveis > row_treino.horas_disponiveis:
                        pesos[1] = row_treino.horas_disponiveis/row.horas_disponiveis
                    elif row.horas_disponiveis == row_treino.horas_disponiveis:
                        pesos[1] = 0.9 # se o horário for igual devemos aumentar o peso padrão
                    else:
                        pesos[1] = row.horas_disponiveis/row_treino.horas_disponiveis

                    current_sum = (sum(np.equal(valores_teste, valores_treino)*pesos)+ pesos[1])/(len(X_teste.columns) - 2)
                    rating = np.around(current_sum, decimals=2)
                    temp_matches[row_treino] = float(format(rating, '.2f'))
                valores_treino = [] #resetamos o valor a cada iteração que a row de treinamento muda

            matches_dict[row] = temp_matches
            temp_matches = {}

        return collections.OrderedDict(sorted(matches_dict.items(), reverse=True))
    
    def filter_ranked_data (self,matches_dict, min_val=0.65):
        results = {}
        for key,value in matches_dict.items():
            filter = {k:v for k,v in value.items() if v >= min_val}
#             print("\nFilter: ", filter)
            ordered_filter = collections.OrderedDict(sorted(filter.items(), reverse=True))  
#             print("\nOrdered Filter: ", ordered_filter)
            for k,v in ordered_filter.items():
                 results[key.user_id] = {k.user_id:v for k,v in ordered_filter.items()}
                    
        return results #retornamos um dicionario com os dados filtrados, isso é o que temos que mandar através da api e tratar
        
    def visualize_recommendations(self,filtered_dict, categoric_df):
    
        for key, value in filtered_dict.items():
            test_data = categoric_df.loc[key]
            columns = categoric_df.columns
            print("-----------------------")
            print("Dado para teste: ")
            print("-----------------------")
            for i in range(len(test_data)):
                print("{0}: {1}".format(columns[i], test_data[i]))
            print("---------------------------------------------")
            print("Recomendações: ")
            print("-----------------------")
            value = collections.OrderedDict(sorted(value.items(), reverse=True)) #ordenando notas
            for k,v in value.items():
                rating = v
                rating_user = categoric_df.loc[k]
                print("Recomendação: %.2f%%" % (rating*100))
                for i in range(len(rating_user)):
                    print("{0}: {1}".format(columns[i], rating_user[i]))
                print("----")

In [213]:
dados_raw = pd.read_json("dados.json")
rank = Ranking(dados_json)
rank.preprocessDataFrame()
X_treino = dados_json[:len(dados_json)-3] 
X_teste  = dados_json[len(dados_json)-3:]
ranked_data = rank.rank_the_data(X_treino, X_teste)
filtered_data = rank.filter_ranked_data(ranked_data) #retorna a % e o id do usuario relacionado
print(filtered_data)

{37: {34: 0.92, 30: 0.75, 29: 0.74, 24: 0.67, 23: 0.75, 8: 0.81, 5: 0.69, 4: 0.74, 2: 0.68}, 36: {33: 0.81, 32: 0.68, 31: 0.74, 25: 0.92, 22: 0.92, 21: 0.74, 20: 0.74, 9: 0.68, 4: 0.69}, 35: {34: 0.68, 30: 0.85, 29: 0.91, 25: 0.74, 24: 0.81, 23: 0.85, 22: 0.74, 9: 0.74, 8: 0.74, 5: 0.91, 4: 0.87}}


In [214]:
dados_raw.iloc[37]

habilidades          [photoshop, illustrator, html, css]
horas_disponiveis                                      4
segmento                                         biotech
tipo_user                                           user
Name: 37, dtype: object

In [215]:
rank.visualize_recommendations(filtered_data, dados_raw)

-----------------------
Dado para teste: 
-----------------------
habilidades: ['photoshop', 'illustrator', 'html', 'css']
horas_disponiveis: 4
segmento: biotech
tipo_user: user
---------------------------------------------
Recomendações: 
-----------------------
Recomendação: 92.00%
habilidades: ['html', 'css', 'photoshop', 'illustrator']
horas_disponiveis: 6
segmento: healthtech
tipo_user: startup
----
Recomendação: 75.00%
habilidades: ['js', 'html', 'css', 'ionic']
horas_disponiveis: 4
segmento: adtech
tipo_user: startup
----
Recomendação: 74.00%
habilidades: ['ionic', 'angular', 'js', 'html', 'css']
horas_disponiveis: 4
segmento: biotech
tipo_user: startup
----
Recomendação: 67.00%
habilidades: ['ionic', 'js', 'html', 'css', 'scss']
horas_disponiveis: 6
segmento: biotech
tipo_user: startup
----
Recomendação: 75.00%
habilidades: ['angular', 'js', 'html', 'css']
horas_disponiveis: 4
segmento: adtech
tipo_user: startup
----
Recomendação: 81.00%
habilidades: ['js', 'html', 'css', 'illu

In [210]:
def generateFullJSON(raw_df,filtered_dict):
    #the JSON should return the following structure:
    jsons      = []
    for key,item in filtered_dict.items():
        user_type = raw_df.loc[:,'tipo_user'][key]
        user_id = key
        skills = raw_df.loc[:,'habilidades'][key]
        market = raw_df.loc[:, 'segmento'][key]
        hours = raw_df.loc[:, 'horas_disponiveis'][key]
        codigo = {"user_id": int(user_id), "user_type": user_type, 
                  "skills":skills,"market":market,"hours":float(hours),"ratings": []}
        
        for k,v in item.items():
            
            guess_user_type = raw_df.loc[:, 'tipo_user'][k]
            guess_user_id = k
            guess_skills = raw_df.loc[:,'habilidades'][k]
            guess_market = raw_df.loc[:, 'segmento'][k]
            guess_hours = raw_df.loc[:, 'horas_disponiveis'][k]
            
            codigo["ratings"].append({"user_id": int(guess_user_id), "user_type": guess_user_type, 
                  "skills":guess_skills,"market":guess_market,"hours":float(guess_hours),"rating":float(v)})
            
        jsons.append(codigo)
    return json.dumps(jsons)
            
        
    

In [211]:
JSON = generateFullJSON(dados_raw, filtered_data)
JSON

'[{"user_id": 37, "user_type": "user", "skills": ["photoshop", "illustrator", "html", "css"], "market": "biotech", "hours": 4.0, "ratings": [{"user_id": 34, "user_type": "startup", "skills": ["html", "css", "photoshop", "illustrator"], "market": "healthtech", "hours": 6.0, "rating": 0.92}, {"user_id": 30, "user_type": "startup", "skills": ["js", "html", "css", "ionic"], "market": "adtech", "hours": 4.0, "rating": 0.75}, {"user_id": 29, "user_type": "startup", "skills": ["ionic", "angular", "js", "html", "css"], "market": "biotech", "hours": 4.0, "rating": 0.74}, {"user_id": 24, "user_type": "startup", "skills": ["ionic", "js", "html", "css", "scss"], "market": "biotech", "hours": 6.0, "rating": 0.67}, {"user_id": 23, "user_type": "startup", "skills": ["angular", "js", "html", "css"], "market": "adtech", "hours": 4.0, "rating": 0.75}, {"user_id": 8, "user_type": "startup", "skills": ["js", "html", "css", "illustrator"], "market": "edtech", "hours": 5.0, "rating": 0.81}, {"user_id": 5, "

In [104]:
pesos ={}

for column in X_treino.columns[2:]:
    pesos[column] = 0
pesos

{'angular': 0,
 'big-data': 0,
 'css': 0,
 'horas_disponiveis': 0,
 'html': 0,
 'illustrator': 0,
 'ionic': 0,
 'js': 0,
 'laravel': 0,
 'machine-learning': 0,
 'photoshop': 0,
 'php': 0,
 'python': 0,
 'scikit-learn': 0,
 'scrum': 0,
 'scss': 0,
 'segmento': 0}

In [102]:
for peso in pesos :
    if peso not in ['segmento','horas_disponiveis']:
        print(peso)

big-data
machine-learning
illustrator
ionic
python
angular
scikit-learn
html
photoshop
css
scss
js
laravel
php
scrum


In [153]:
a = 0.9400000000000
a = float(format(a, '.2f'))

In [154]:
type(a)

float